In [1]:
# Template Notebook for Engagement Classifcation Model

# ToDo/Models to run here: https://docs.google.com/document/d/18EJpFesMEl9Q7C1AZzDeq6uy7c8tDMvEJ_j58-MmiBI/edit?usp=sharing
# Use p8_data_processed.csv here: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [26]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from keras.layers import Dense, Activation
from keras.models import Sequential

In [3]:
# Load Data
# Warning: this will probably take some time
# Adjust file path based on your computer

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_data_processed.csv'
data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,participant,session_num,session_date,timestamp,engagement,activity,skill,difficulty,aptitude,games_total,...,op_p2_pose_y9,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,8.0,1.0,2018-06-07,0.000000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
1,8.0,1.0,2018-06-07,0.033333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
2,8.0,1.0,2018-06-07,0.066667,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
3,8.0,1.0,2018-06-07,0.100000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
4,8.0,1.0,2018-06-07,0.133333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1


In [5]:
# Drop Irrelevant Columns

# transcriptions
data = data.drop(columns=['transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'])

# raw ros messages
data = data.drop(columns=['ros_PARTICIPANT_STATE', 'ros_ROBOT_STATE'])

# more for counting purposes
data = data.drop(columns=['game_start', 'game_correct', 'game_incorrect', 'mistake_made'])

data = data.drop(columns=['participant', 'session_date'])

# use one-hot encoded difficulty and skill, don't use activity for now
data = data.drop(columns=['difficulty', 'skill', 'activity', 'of_face_id'])

In [6]:
# Open-Face/Open-Pose Columns? 
# Feature Dictionary Documentation here: https://docs.google.com/document/d/1RSygoLwsM1PKEIOoqDOaEo3lRBZBLZJuzbMSY5K_6zA/edit?usp=sharing

only_ofop = []
performance = []
for i in data.columns:
    # open face/open pose columns begin with of_ or op_
    if 'op_' in i or 'of_' in i:
        only_ofop.append(i)
    else:
        performance.append(i)
        
only_ofop.remove('of_timestamp')

In [7]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

1.0    139463
0.0    120473
Name: engagement, dtype: int64

1 0.5365282223316509
0 0.4634717776683491


### Step 1: Set Up Feature Sets

1. All OF/OP + Performance
2. Performance ONLY
3. All OF/OP ONLY
4. Performance + Handpicked OF/OP (from Caitlyn's intuition)
5. Performance + Selected OF/OP (from Feature Analysis) 

Note: keep timestamp and session_num until right before running the model in all feature sets

In [8]:
# Feature Set 1: All OF/OP + Performance

FS1 = data

In [ ]:
# Feature Set 2: Performance ONLY

print(performance)

FS2 = data[performance]

In [ ]:
# Feature Set 3: OF/OP ONLY  

only_ofop.append('timestamp')
only_ofop.append('session_num')
only_ofop.append('engagement')

FS3 = data[only_ofop]

In [ ]:
# Feature Set 4: Performance + Handpicked OF/OP

hand_picked = ['confidence', 'success', 'gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']
for i,c in enumerate(hand_picked):
    hand_picked[i] = 'of_' + c
hand_picked.append('op_Number of People')

for i in performance:
    hand_picked.append(i)

FS4 = data[hand_picked]

In [ ]:
# Feature Set 5: Performance + Selected OF/OP

selected_ofop = ['of_AU04_c',   'of_AU06_c',    'of_AU07_c',    'of_AU14_c',    'of_AU15_c',    'of_AU25_c',    'of_confidence',    'of_frame',    'of_success',    'of_timestamp', 'op_p1_face_c25',    'op_p1_face_c43',    'op_p1_face_c44',    'op_p1_face_c45',    'op_p1_face_c46',    'op_p1_face_y7',    'op_p1_face_y8',    'op_p1_pose_c0',    'op_p1_pose_c15',    'op_p1_pose_c16']

for i in performance:
    selected_ofop.append(i)

FS5 = data[selected_ofop]

### Step 2: Train Test Split (for chosen Feature Set)

1. Random Shuffle 70-30
2. Train on Earlier Sessions, Test on Later Sessions 70-30

In [9]:
# CHOOSE FEATURE SET

FS = FS1

In [10]:
# Optional: Shuffle Data if not temporal model

FS = FS.sample(frac=1).reset_index(drop=True)

In [11]:
# Train-Test Split 1: Random Shuffle 70-30

y1 = FS['engagement']
X1 = FS.drop(columns=['timestamp', 'session_num', 'engagement'])

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [37]:
# Train Test Split 2: Train on Earlier Sessions, Test on Later Sessions

session_threshold = 8 # train on sessions <= 8, test on sessions > 8

train = FS[(FS['session_num'] <= 8.0)] 
test = FS[(FS['session_num'] > 8)]

# drop columns needed for split
train2 = train.drop(columns=['session_num', 'timestamp'])
test2 = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train2.drop(columns=['engagement'])
y_train2 = train2['engagement']

X_test2 = test2.drop(columns=['engagement'])
y_test2 = test2['engagement']

### Step 3: Neural Network Model

In [17]:
num_epochs = 20
size_batch = 256

In [14]:
# NN Model 1 for FS1 (all OF/OP)

model = Sequential()

model.add(Dense(units=128, input_dim=len(X_train1.columns))) # Should have 1554 features in FS1
model.add(Activation('relu'))

model.add(Dense(units=64))
model.add(Activation('relu'))

model.add(Dense(units=32))
model.add(Activation('relu'))

model.add(Dense(units=1))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               199040    
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_3 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
__________

In [15]:
model.compile(loss='binary_crossentropy', # specify crossentropy cost function 
              optimizer='sgd', # stochastic gradient descent
              metrics=['accuracy'])

#### Train-Test Split 1

In [18]:
model.fit(X_train1, y_train1, epochs=num_epochs, batch_size=size_batch, verbose=0, validation_split = .1)

In [19]:
loss_and_metrics = model.evaluate(X_test1, y_test1, batch_size=128)
print(loss_and_metrics)

77981/77981 [==============================] - 5s 68us/step
[0.2251311304949324, 0.9042587296852153]


In [24]:
scores = model.predict_proba(X_test1)
pred = model.predict_classes(X_test1)

In [27]:
print("AUC:",roc_auc_score(y_test1, scores)) 
print("Accuracy:",accuracy_score(y_test1, pred))
print(metrics.confusion_matrix(y_test1, pred))
print(metrics.classification_report(y_test1, pred))

AUC: 0.9713936654596413
Accuracy: 0.9042587296905656
[[33686  2642]
 [ 4824 36829]]
              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90     36328
         1.0       0.93      0.88      0.91     41653

   micro avg       0.90      0.90      0.90     77981
   macro avg       0.90      0.91      0.90     77981
weighted avg       0.91      0.90      0.90     77981



#### Train-Test Split 2

In [34]:
model.fit(X_train2, y_train2, epochs=num_epochs, batch_size=size_batch, verbose=0, validation_split = .1)

In [38]:
loss_and_metrics = model.evaluate(X_test2, y_test2, batch_size=128)
print(loss_and_metrics)

105092/105092 [==============================] - 7s 71us/step
[0.2784716728270049, 0.8841586419518136]


In [39]:
scores = model.predict_proba(X_test2)
pred = model.predict_classes(X_test2)

In [40]:
print("AUC:",roc_auc_score(y_test2, scores)) 
print("Accuracy:",accuracy_score(y_test2, pred))
print(metrics.confusion_matrix(y_test2, pred))
print(metrics.classification_report(y_test2, pred))

AUC: 0.9674745298642521
Accuracy: 0.8841586419518136
[[62076  2501]
 [ 9673 30842]]
              precision    recall  f1-score   support

         0.0       0.87      0.96      0.91     64577
         1.0       0.92      0.76      0.84     40515

   micro avg       0.88      0.88      0.88    105092
   macro avg       0.90      0.86      0.87    105092
weighted avg       0.89      0.88      0.88    105092

